# I will create a Linear Regression model

In [1]:
# Import the necessary libraries 
import numpy as np
import pandas as pd 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, log_loss
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

In [2]:

# Read train features
train_aug = pd.read_csv("../Data/train_labels_augmented.csv")
train_aug.set_index(train_aug.sample_id, inplace=True)
train_aug


,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,...,3_ratio_peak_abund,3_ratio_meanstd_abund,4_ratio,4_ratio_peak_temp,4_ratio_peak_abund,4_ratio_meanstd_abund,5_ratio,5_ratio_peak_temp,5_ratio_peak_abund,5_ratio_meanstd_abund
sample_id,,,,,,,,,,,,,,,,,,,,,
S0000,S0000,0,0,0,0,0,0,0,0,1,...,0.024563,0.009736,28.0,35.420,0.010877,0.009101,19.0,105.458,0.006600,0.002841
S0001,S0001,0,1,0,0,0,0,0,0,0,...,0.033209,0.024006,16.0,497.404,0.093082,0.020585,18.0,-60.370,0.030234,0.019214
S0002,S0002,0,0,0,0,0,1,0,0,0,...,0.294738,0.075850,16.0,545.304,0.040446,0.013012,1.0,145.673,0.004003,0.001801
S0003,S0003,0,1,0,1,0,0,0,0,1,...,0.260027,0.061956,28.0,593.222,0.081701,0.031606,16.0,593.222,0.083077,0.028375
S0004,S0004,0,0,0,1,0,1,1,0,0,...,0.238290,0.082294,16.0,471.733,0.084732,0.026218,44.0,333.273,0.029998,0.013378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S0749,S0749,0,0,0,0,0,0,0,0,0,...,0.270481,0.187613,28.0,523.057,0.130122,0.079981,16.0,228.295,0.082464,0.057972
S0750,S0750,0,0,0,0,0,0,1,0,0,...,0.403334,0.152057,32.0,906.311,0.158759,0.122547,16.0,664.846,0.079410,0.041182
S0751,S0751,0,0,0,0,0,0,0,1,0,...,0.261788,0.143669,28.0,-60.026,0.065964,0.068941,32.0,96.019,0.032240,0.019636


### I perform a train/test split on our overall training dataset and separate the predictors (columns after the ions) and the predictions (columns of ions). Then I create a KNN Classification model that takes into consideration the 3 closest neighbors and uses them to predict whether the ions are present.

In [3]:
# Separate inputs from outputs
X = train_aug.iloc[:,12:]
y = train_aug.iloc[:,1:11]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg = LinearRegression().fit(X_train, y_train)

y_pred = reg.predict(X_test)

### Our predictions for the test set come out as an array, so we need to put it in a dataframe and put back the column labels.

In [4]:
y_pred_df=pd.DataFrame(y_pred, columns=['basalt','carbonate','chloride','iron_oxide','oxalate','oxychlorine','phyllosilicate','silicate','sulfate','sulfide'])
y_pred_df

,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
0,0.329643,0.186382,-0.014385,-0.070428,0.017219,-0.046532,0.363293,0.411213,-0.015432,-0.027554
1,-0.049999,-0.023191,0.086668,-0.012241,0.618995,0.299731,0.154567,0.039877,-0.113506,0.109458
2,0.232035,0.207018,0.331556,0.337544,-0.137613,0.400736,0.407270,0.025281,0.624387,0.117504
3,0.023808,0.135550,0.083464,0.142800,0.035346,0.192978,0.444705,0.081867,0.131780,0.080139
4,0.209831,0.153333,0.067330,0.331916,-0.091808,0.137115,0.412706,0.183939,0.304215,0.071548
...,...,...,...,...,...,...,...,...,...,...
146,0.141504,0.054522,0.291958,0.214988,0.055775,0.379488,0.345023,0.179180,0.270487,0.141375
147,0.100379,0.145160,-0.084660,0.340982,0.002958,0.215798,0.253401,0.244893,0.021465,0.019874
148,0.004618,0.213222,0.174403,0.121192,-0.057978,0.238275,0.258130,-0.066388,0.207379,0.007826
149,0.296168,0.315258,0.021267,0.316699,-0.024514,0.221165,0.315866,0.302529,0.410742,0.093077


### Comparing what we got with what we expected to get.

In [5]:
y_test=y_test.reset_index()
y_test.drop('sample_id', axis=1, inplace=True)
y_test_int=y_test.replace(0,"0").replace(1,"1")
y_test

,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0
2,1,1,0,1,0,1,1,1,1,0
3,0,0,1,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
146,0,0,0,0,0,1,0,0,0,1
147,0,0,0,0,0,0,1,0,0,0
148,0,0,0,0,0,1,0,0,0,0
149,1,1,0,1,0,1,1,1,1,0


### Might be worth rounding to make the model easier to analyze.

In [6]:
y_pred_integers = y_pred_df.round(0)
y_pred_integers= y_pred_integers.replace(-0.0,"0").replace(0.0,"0").replace(1.0,"1")
y_pred_df = y_pred_integers
y_pred_df

,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
146,0,0,0,0,0,0,0,0,0,0
147,0,0,0,0,0,0,0,0,0,0
148,0,0,0,0,0,0,0,0,0,0
149,0,0,0,0,0,0,0,0,0,0


### I think we can already tell this is going to be bad, but we can see how many rows of our predictions were identical to the corresponding row of our expectation.

In [7]:
count=0
for i in range(len(y_test_int)-1):
    if y_test_int.iloc[i].equals(y_pred_integers.iloc[i]):
        count=count+1

### Next we calculate the percent accuracy for each sample. (Just check whether we got all columns correct for each sample using this model.)

In [8]:
count/(len(y_test))*100

19.205298013245034

### 19% accuracy is not great, but we should not be surprised with a linear model. Also that's for the entire row being correct. Let's try to examine how good this model is for the presence of each ion separately.

#### Basalt:

In [9]:
basaltcount=0
for i in range(len(y_test)-1):
    if y_test_int.basalt[i]==(y_pred_df.basalt[i]):
        basaltcount=basaltcount+1
basalt = basaltcount/len(y_test)*100
basalt

86.75496688741721

#### Carbonate

In [10]:
carbonatecount=0
for i in range(len(y_test)-1):
    if y_test_int.carbonate[i]==(y_pred_df.carbonate[i]):
        carbonatecount=carbonatecount+1
carbonate = carbonatecount/len(y_test)*100
carbonate

90.06622516556291

#### Chloride

In [11]:
chloridecount=0
for i in range(len(y_test)-1):
    if y_test_int.chloride[i]==(y_pred_df.chloride[i]):
        chloridecount=chloridecount+1
chloride = chloridecount/len(y_test)*100
chloride

86.09271523178808

#### Iron Oxide

In [12]:
IOcount=0
for i in range(len(y_test)-1):
    if y_test_int.iron_oxide[i]==(y_pred_df.iron_oxide[i]):
        IOcount=IOcount+1
ironoxide = IOcount/len(y_test)*100
ironoxide

78.1456953642384

#### Oxalate

In [13]:
Oxcount=0
for i in range(len(y_test)-1):
    if y_test_int.oxalate[i]==(y_pred_df.oxalate[i]):
        Oxcount=Oxcount+1
oxalate = Oxcount/len(y_test)*100
oxalate

98.01324503311258

#### Oxychlorine

In [14]:
Oxycount=0
for i in range(len(y_test)-1):
    if y_test_int.oxychlorine[i]==(y_pred_df.oxychlorine[i]):
        Oxycount=Oxycount+1
oxychlorine = Oxycount/len(y_test)*100
oxychlorine

79.47019867549669

#### Phyllosilicate

In [15]:
Phycount=0
for i in range(len(y_test)-1):
    if y_test_int.phyllosilicate[i]==(y_pred_df.phyllosilicate[i]):
        Phycount=Phycount+1
phyllosilicate = Phycount/len(y_test)*100
phyllosilicate

66.88741721854305

#### Silicate

In [16]:
Silcount=0
for i in range(len(y_test)-1):
    if y_test_int.silicate[i]==(y_pred_df.silicate[i]):
        Silcount=Silcount+1
silicate = Silcount/len(y_test)*100
silicate

86.09271523178808

#### Sulfate

In [17]:
Sulfatecount=0
for i in range(len(y_test)-1):
    if y_test_int.sulfate[i]==(y_pred_df.sulfate[i]):
        Sulfatecount=Sulfatecount+1
sulfate = Sulfatecount/len(y_test)*100
sulfate

81.45695364238411

#### Sulfide

In [18]:
Sulfidecount=0
for i in range(len(y_test)-1):
    if y_test_int.sulfide[i]==(y_pred_df.sulfide[i]):
        Sulfidecount=Sulfidecount+1
sulfide = Sulfidecount/len(y_test)*100
sulfide

96.68874172185431

### Now the average of these percent accuracies is slightly better.

In [19]:
(basalt + carbonate + chloride + ironoxide + oxalate + oxychlorine + phyllosilicate + silicate + sulfate + sulfide)/10

84.96688741721854

### Honestly, much better than I thought we'd do.